In [1]:
%matplotlib inline

In [8]:
import os
import sys
import numpy as np
import matplotlib

if not sys.version_info.major is 3:
    print('PyRadarKit requires Python 3')
    raise

In [9]:
import chart
import radarkit

In [10]:
matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.serif'] = ['Arial']
matplotlib.rcParams['font.sans-serif'] = ['System Font', 'Verdana', 'Arial']
matplotlib.rc('text', usetex=False)

In [11]:
sweep = radarkit.read('/Users/boonleng/Downloads/PX-20170220-050706-E2.4-Z.nc')
if sweep is None:
    print('Unable to continue.\n')
    raise

In [25]:
for product in sweep['products']:
    symbol = product['symbol']
    if not 'R' is product['symbol']:
        continue
    name = product['name']
    unit = product['unit']
    data = product['data'][:, 0:ng]
data = np.nan_to_num(data)

In [26]:
def rho2ind1(values):
    m3 = values > 0.93
    m2 = np.logical_and(values > 0.7, ~m3)
    m1 = values <= 0.7
    index = np.zeros(values.shape);
    index[m3] = ((values[m3] - 0.93) * 1000.0) + 106
    index[m2] = ((values[m2] - 0.7) * 300) + 37
    index[m1] = values[m1] * 52.8751
    return np.round(index)

def rho2ind2(values):
    m3 = values > 0.93
    m2 = np.logical_and(values > 0.7, ~m3)
    index = values * 52.8751
    index[m2] = values[m2] * 300.0 - 173.0
    index[m3] = values[m3] * 1000.0 - 824.0
    return np.round(index)

def rho2ind3(values):
    m3 = values > 0.93
    m2 = np.logical_and(values > 0.7, ~m3)
    m1 = values <= 0.7
    w = 1000.0 * m3 + 300.0 * m2 + 52.8751 * m1
    b = -824.0 * m3 - 173.0 * m2
    index = np.round(w * values + b)
    return index

In [27]:
%timeit ind1a = rho2ind1(data)
%timeit ind1b = rho2ind1(data)
%timeit ind2a = rho2ind2(data)
%timeit ind2b = rho2ind2(data)
%timeit ind3a = rho2ind3(data)
%timeit ind3b = rho2ind3(data)

2.17 ms ± 8.69 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.17 ms ± 4.92 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.86 ms ± 2.41 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.86 ms ± 3.13 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
2.56 ms ± 31.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.55 ms ± 2.86 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
